In [1]:
!ls

create_metadata_file.ipynb	    non_clinical_metrics_metadata.json
full_role_based_metric_access.json


In [17]:
import json
import pandas as pd

# Load JSON files
with open("non_clinical_metrics_metadata.json") as f:
    metadata = json.load(f)

with open("full_role_based_metric_access.json") as f:
    role_access = json.load(f)

# Step 1: Flatten metadata into DataFrame
records = []
for sheet_name, metrics in metadata.items():
    for metric in metrics:
        record = {
            "KPI": metric["Column Name"],
            "Data Type": metric["Data Type"],
            "Description": metric["Description"],
            "Granularity": metric["Granularity"],
            "Calculation": metric["Calculation"],
            "Source Table/Sheet": sheet_name,
        }
        records.append(record)

df = pd.DataFrame(records)

# Step 2: Initialize access columns
df["Non Clinical access"] = 0
df["Admin Access"] = 0
df["Manager Access"] = 0

# Step 3: Update access columns based on role_access.json
role_column_map = {
    "non_clinical_staff": "Non Clinical access",
    "administrative_staff": "Admin Access",
    "managers": "Manager Access",
}

for role, role_data in role_access.items():
    access_col = role_column_map.get(role)
    print(role, access_col)
    if not access_col:
        continue
    for metric in role_data["metrics"]:
        col_name = metric["Column Name"]
        sheet = metric.get("source_sheet", "")
        # Match both column name and sheet to be safe
        mask = (df["KPI"] == col_name) & (df["Source Table/Sheet"] == sheet)
        df.loc[mask, access_col] = 1

# Step 4: Add Department Name & Description if available (optional, example placeholders)
df["Department Name"] = df["Source Table/Sheet"]
df["Department Description(optional)"] = None  # You can enrich if needed

# Output
print(df)


non_clinical_staff Non Clinical access
administrative_staff Admin Access
managers Manager Access
                                  KPI Data Type  \
0                                date    object   
1                       department_id    object   
2       patient_arrival_count_monthly     int64   
3         patient_arrival_count_daily    object   
4    avg_waiting_time_minutes_monthly   float64   
..                                ...       ...   
138        patient_satisfaction_score   float64   
139         avg_resolution_time_hours   float64   
140        patient_recommendation_pct   float64   
141         avg_response_time_minutes   float64   
142  patient_education_completion_pct   float64   

                                           Description           Granularity  \
0                             Date of metric recording                 Daily   
1           Unique identifier for emergency department      Department-level   
2        Total number of patient arrivals in the m

In [18]:
access_col

'Manager Access'

In [19]:
role_access

{'non_clinical_staff': {'role_description': 'Front-desk staff, pharmacy/radiology assistants, janitors, and general support staff focused on daily task execution.',
  'metrics': [{'Column Name': 'patient_arrival_count_monthly',
    'Data Type': 'int64',
    'Description': 'Total number of patient arrivals in the month',
    'Granularity': 'Monthly',
    'Calculation': 'Sum of patient arrivals for the month',
    'source_sheet': 'emergency_department_metrics'},
   {'Column Name': 'patient_arrival_count_daily',
    'Data Type': 'object',
    'Description': 'Daily count of patient arrivals (JSON format)',
    'Granularity': 'Daily (nested)',
    'Calculation': 'Aggregated from daily patient logs, may include fractional values due to smoothing or averaging',
    'source_sheet': 'emergency_department_metrics'},
   {'Column Name': 'bed_utilization_rate',
    'Data Type': 'float64',
    'Description': 'Emergency bed utilization rate (%)',
    'Granularity': 'Monthly',
    'Calculation': '(Bed

In [20]:
df

,KPI,Data Type,Description,Granularity,Calculation,Source Table/Sheet,Non Clinical access,Admin Access,Manager Access,Department Name,Department Description(optional)
0,date,object,Date of metric recording,Daily,Direct from ED logs,emergency_department_metrics,0,0,1,emergency_department_metrics,None
1,department_id,object,Unique identifier for emergency department,Department-level,Direct from hospital system,emergency_department_metrics,0,0,1,emergency_department_metrics,None
2,patient_arrival_count_monthly,int64,Total number of patient arrivals in the month,Monthly,Sum of patient arrivals for the month,emergency_department_metrics,1,0,1,emergency_department_metrics,None
3,patient_arrival_count_daily,object,Daily count of patient arrivals (JSON format),Daily (nested),"Aggregated from daily patient logs, may includ...",emergency_department_metrics,1,0,1,emergency_department_metrics,None
4,avg_waiting_time_minutes_monthly,float64,Average waiting time (arrival to first treatme...,Monthly,Average wait time per month,emergency_department_metrics,0,0,1,emergency_department_metrics,None
...,...,...,...,...,...,...,...,...,...,...,...
138,patient_satisfaction_score,float64,Overall patient satisfaction score (typically ...,Hospital-level daily,Aggregated average score from surveys,patient_experience_metrics,0,1,1,patient_experience_metrics,None
139,avg_resolution_time_hours,float64,Average time taken to resolve patient issues (...,Hospital-level daily,Calculated as average resolution time per case,patient_experience_metrics,0,1,1,patient_experience_metrics,None
140,patient_recommendation_pct,float64,Percentage of patients who would recommend the...,Hospital-level daily,Calculated from survey data on patient recomme...,patient_experience_metrics,0,0,1,patient_experience_metrics,None
141,avg_response_time_minutes,float64,Average time taken to respond to patient inqui...,Hospital-level daily,Average time from issue raised to first response,patient_experience_metrics,0,0,1,patient_experience_metrics,None


In [21]:
# Step 3: Mark access from role-based definitions
role_column_map = {
    "non_clinical_staff": "Non Clinical access",
    "admin": "Admin Access",
    "manager": "Manager Access",
}

all_role_kpis = set()

for role, role_data in role_access.items():
    access_col = role_column_map.get(role)
    if not access_col:
        continue
    for metric in role_data["metrics"]:
        col_name = metric["Column Name"]
        sheet = metric.get("source_sheet", "")
        all_role_kpis.add((col_name, sheet))
        # Match by both name and sheet
        mask = (df["KPI"] == col_name) & (df["Source Table/Sheet"] == sheet)
        df.loc[mask, access_col] = 1


# Step 4: Validation checks

## A. Check for KPIs with NO access
df["Total Access"] = (
    df["Non Clinical access"]
    + df["Admin Access"]
    + df["Manager Access"]
)

missing_access = df[df["Total Access"] == 0]
if not missing_access.empty:
    print("❌ The following metrics have no role access defined:")
    print(missing_access[["KPI", "Source Table/Sheet"]])
else:
    print("✅ All rows have at least one role with access.")

## B. Check for any KPIs mentioned in role JSON that are not in metadata
metadata_keys = set(zip(df["KPI"], df["Source Table/Sheet"]))
undefined_metrics = all_role_kpis - metadata_keys

if undefined_metrics:
    print("\n❌ These KPIs exist in role JSON but are MISSING in metadata:")
    for item in undefined_metrics:
        print(f" - Column: {item[0]}, Sheet: {item[1]}")
else:
    print("✅ All role-based KPIs are present in metadata.")

# Clean up
df.drop(columns=["Total Access"], inplace=True)


✅ All rows have at least one role with access.
✅ All role-based KPIs are present in metadata.


In [22]:
missing_access

,KPI,Data Type,Description,Granularity,Calculation,Source Table/Sheet,Non Clinical access,Admin Access,Manager Access,Department Name,Department Description(optional),Total Access


In [24]:
df.to_csv("non_clinical_metrics_access.csv", index=False)